In [1]:
# import pandas as pd
# import numpy as np
# from sklearn import preprocessing
# from sklearn.model_selection import train_test_split
# from sklearn.svm import SVC
# from sklearn.linear_model import LogisticRegression
# from keras.models import Sequential
# from keras.layers import Dense, Dropout
# from keras.optimizers import SGD, RMSprop, Adam
# from keras.models import load_model

# import tensorflow as tf
# from neupy import algorithms
# from sklearn.metrics import mean_squared_error, confusion_matrix
# from sklearn.metrics import multilabel_confusion_matrix
# from math import sqrt
# from sklearn.tree import DecisionTreeClassifier
# from sklearn import metrics

# # 用隨機森林做Feature Selection
# from sklearn.ensemble import ExtraTreesClassifier
# from sklearn.feature_selection import SelectFromModel

# # ROC Curve
# from scipy import interp
# from itertools import cycle
# from sklearn.metrics import roc_curve, auc

# import re
# from datetime import date
# import datetime

# import matplotlib.pyplot as plt
# import seaborn as sns
# %matplotlib inline

Using TensorFlow backend.


In [61]:
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas()

/home/barry/miniconda3/envs/bertenv/lib/python3.7/site-packages/tqdm/std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


# 1 匯入資料＆前處理

In [3]:
df = pd.read_csv('data/TCRI_2008_2017.csv', encoding="utf-8")
df = df.sort_values(by=["年月","公司"])   # 先日期排序 再公司名排序
df = df.reset_index(drop=True)           # index重新排序
print(df.shape)
df.head(2)

(2337, 18)


,公司,年月,TCRI,總分,規模,獲利,安全,效率,CFO,CFO不一致,利息支出率,資產報酬%,營業利益%,借款息%,平均收款天增減,平均售貨天增減,借款依存度%,營收
0,1101 台泥,2008/12/31,3,622.0,140.0,25.0,61.0,85.0,80.43,NaN,3.51,2.93,6.64,3.72,-2.0,-2.0,88.0,785.0
1,1201 味全,2008/12/31,5,621.0,105.0,38.0,74.0,78.0,10.75,NaN,0.54,4.62,4.04,3.27,-2.0,1.0,63.0,202.0


In [4]:
df.describe()

,總分,規模,獲利,安全,效率,CFO,利息支出率,資產報酬%,營業利益%,借款息%,平均收款天增減,平均售貨天增減,借款依存度%,營收
count,2335.000000,2335.000000,2335.000000,2335.000000,2335.000000,2335.000000,2335.000000,2335.000000,2335.000000,2334.000000,2301.000000,2301.000000,2335.000000,2335.000000
mean,483.451820,68.855675,33.986296,68.811135,39.749465,53.032857,4.941863,3.861251,-1.701443,2.294636,-0.085615,206.398522,71.110921,413.813704
std,293.200973,49.174420,42.023821,60.271720,40.871758,276.646230,139.468763,8.793073,168.423238,1.605054,82.285696,4296.069004,178.469003,1193.327092
min,-1465.000000,0.000000,-60.000000,-888.000000,-40.000000,-190.300000,0.000000,-75.930000,-7928.410000,0.000000,-2280.000000,-8994.000000,-736.000000,0.000000
25%,366.000000,23.000000,0.000000,67.000000,19.000000,0.430000,0.160000,0.390000,-0.065000,1.460000,-4.000000,-5.000000,15.000000,25.000000
50%,529.000000,59.000000,33.000000,87.000000,49.000000,4.950000,0.560000,3.970000,4.620000,2.020000,1.000000,0.000000,42.000000,66.000000
75%,670.000000,122.000000,65.000000,100.000000,70.500000,18.125000,1.420000,7.960000,11.030000,2.830000,6.000000,7.000000,77.000000,209.000000
max,1020.000000,140.000000,120.000000,100.000000,100.000000,5796.730000,6725.280000,95.780000,135.080000,22.460000,2197.000000,90475.000000,4416.000000,12137.000000


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2337 entries, 0 to 2336
Data columns (total 18 columns):
公司         2337 non-null object
年月         2337 non-null object
TCRI       2336 non-null object
總分         2335 non-null float64
規模         2335 non-null float64
獲利         2335 non-null float64
安全         2335 non-null float64
效率         2335 non-null float64
CFO        2335 non-null float64
CFO不一致     286 non-null object
利息支出率      2335 non-null float64
資產報酬%      2335 non-null float64
營業利益%      2335 non-null float64
借款息%       2334 non-null float64
平均收款天增減    2301 non-null float64
平均售貨天增減    2301 non-null float64
借款依存度%     2335 non-null float64
營收         2335 non-null float64
dtypes: float64(14), object(4)
memory usage: 328.8+ KB


## ※匯入TSE產業別代碼

In [12]:
tse = pd.read_csv('data/TSE_category.csv', encoding="utf-8")
print(tse.shape)
tse.head(2)

(240, 2)


,公司簡稱,TSE 產業別
0,2025 千興,10
1,1453 大將,4


In [13]:
tse.set_index('公司簡稱').to_dict()
tse_dict = tse.set_index('公司簡稱')['TSE 產業別'].to_dict()

In [14]:
tse_no = []
for company in df['公司']:
    tse_no.append(tse_dict[company])
df['TSE產業別'] = tse_no
df.head(2)

,公司,年月,TCRI,總分,規模,獲利,安全,效率,CFO,CFO不一致,利息支出率,資產報酬%,營業利益%,借款息%,平均收款天增減,平均售貨天增減,借款依存度%,營收,TSE產業別
0,1101 台泥,2008/12/31,3,622.0,140.0,25.0,61.0,85.0,80.43,NaN,3.51,2.93,6.64,3.72,-2.0,-2.0,88.0,785.0,1
1,1201 味全,2008/12/31,5,621.0,105.0,38.0,74.0,78.0,10.75,NaN,0.54,4.62,4.04,3.27,-2.0,1.0,63.0,202.0,2


In [15]:
df.to_csv('data/TCRI_2008_2017_tse.csv', index=False)

## 1-1 調整「CFO不一致」欄位

In [6]:
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas()

In [62]:
df = pd.read_csv('data/TCRI_2008_2017_tse.csv', encoding="utf-8")
df = df.sort_values(by=["年月","公司"])   # 先日期排序 再公司名排序
df = df.reset_index(drop=True)           # index重新排序
print(df.shape)
df.head(2)

(2337, 19)


,公司,年月,TCRI,總分,規模,獲利,安全,效率,CFO,CFO不一致,利息支出率,資產報酬%,營業利益%,借款息%,平均收款天增減,平均售貨天增減,借款依存度%,營收,TSE產業別
0,1101 台泥,2008/12/31,3,622.0,140.0,25.0,61.0,85.0,80.43,NaN,3.51,2.93,6.64,3.72,-2.0,-2.0,88.0,785.0,1
1,1201 味全,2008/12/31,5,621.0,105.0,38.0,74.0,78.0,10.75,NaN,0.54,4.62,4.04,3.27,-2.0,1.0,63.0,202.0,2


In [63]:
# 將「CFO不一致」欄位的值調整，空值(Nan)與minus sign(-)改為0，plus sign(+)改為1
df['CFO不一致'] = df['CFO不一致'].fillna(0)
df['CFO不一致'] = df['CFO不一致'].replace('+', 1)
df['CFO不一致'] = df['CFO不一致'].replace('-', 0)

## 1-2 調整「TCRI」欄位

In [64]:
# 將TCRI欄位中"-"資料刪掉；D表示違約，另外處理（先暫時改成0）
index_to_drop = df[df['TCRI'] == '-'].index     # 把 df['TCRI'] == '-' 的index 挑出來
df = df.drop(index=index_to_drop, axis=1)       # 刪除指定index 整個row
df['TCRI'] = df['TCRI'].replace('D', 0)        # D表示違約，另外處理（先暫時改成0）

# TCRI原本有非數值的資料，因此原始的型態並非float64，因此需要轉型
df['TCRI'] = df['TCRI'].astype(float)
print(df.shape)
df.head(2)

(2309, 19)


,公司,年月,TCRI,總分,規模,獲利,安全,效率,CFO,CFO不一致,利息支出率,資產報酬%,營業利益%,借款息%,平均收款天增減,平均售貨天增減,借款依存度%,營收,TSE產業別
0,1101 台泥,2008/12/31,3.0,622.0,140.0,25.0,61.0,85.0,80.43,0,3.51,2.93,6.64,3.72,-2.0,-2.0,88.0,785.0,1
1,1201 味全,2008/12/31,5.0,621.0,105.0,38.0,74.0,78.0,10.75,0,0.54,4.62,4.04,3.27,-2.0,1.0,63.0,202.0,2


## 1-3 去除空值與非數值型資料

In [65]:
# 有 空值 NaN 的 row橫列 都刪掉
df.dropna(axis=0, inplace=True)
print(df.shape)

(2288, 19)


## 1-4 去除「規模」、「獲利」、「安全」、「效率」等4個欄位

In [66]:
df = df.drop(['規模', '獲利', '安全', '效率'], axis=1)
print(df.shape)
df = df.reset_index(drop=True)           # index重新排序
df.tail(2)

(2288, 15)


,公司,年月,TCRI,總分,CFO,CFO不一致,利息支出率,資產報酬%,營業利益%,借款息%,平均收款天增減,平均售貨天增減,借款依存度%,營收,TSE產業別
2286,9943 好樂迪,2017/12/29,5.0,711.0,5.20,0,0.00,15.93,17.41,6.53,1.0,0.0,0.0,29.0,16
2287,9946 三發地產,2017/12/29,7.0,243.0,-8.71,1,7.49,3.83,20.42,2.51,9.0,1437.0,102.0,17.0,14


## ※合併輿情風險分數

In [67]:
news_score = pd.read_csv("data/company_year_prob_sum.csv", encoding="utf-8")
news_score["c_0"] = news_score["c_0"].astype(float)  # 讀取後將欄位弄成float
news_score["c_1"] = news_score["c_1"].astype(float)
news_score["c_2"] = news_score["c_2"].astype(float)
news_score["c_3"] = news_score["c_3"].astype(float)
news_score["c_4"] = news_score["c_4"].astype(float)
print(news_score.shape)
news_score.tail(2)

(2400, 7)


,company,date,c_0,c_1,c_2,c_3,c_4
2398,9946 三發地產,2016,0.000002,1.470138e-13,3.596562e-07,4.796958e-23,1.999998e+00
2399,9946 三發地產,2017,1.000000,1.433146e-20,1.467837e-13,2.940595e-25,2.578725e-17


In [68]:
# 把 c_0~c_4都等於0 的index 挑出來
index_to_drop = news_score[(news_score['c_0'] == 0.0) &
                           (news_score['c_1'] == 0.0) &
                           (news_score['c_2'] == 0.0) &
                           (news_score['c_3'] == 0.0) &
                           (news_score['c_4'] == 0.0)].index     # 把 c_0~c_4都等於0 的 row的 index 挑出來
print(len(index_to_drop))
index_to_drop

1972


Int64Index([   0,    1,    2,    5,    6,    7,    8,    9,   10,   11,
            ...
            2388, 2389, 2390, 2391, 2392, 2393, 2394, 2395, 2396, 2397],
           dtype='int64', length=1972)

In [69]:
news_score = news_score.drop(index=index_to_drop, axis=1)       # 刪除指定index 整個row
news_score = news_score.reset_index(drop=True)           # index重新排序
print(news_score.shape)
news_score.tail(2)

(428, 7)


,company,date,c_0,c_1,c_2,c_3,c_4
426,9946 三發地產,2016,0.000002,1.470138e-13,3.596562e-07,4.796958e-23,1.999998e+00
427,9946 三發地產,2017,1.000000,1.433146e-20,1.467837e-13,2.940595e-25,2.578725e-17


#### 因為df 的row數 跟 news_score不一樣
#### 比對 company和year 再把 news_score表格的欄位 填入 df 表格

In [70]:
# # M106陳柏霖寫法

# risk1 = []
# risk2 = []
# risk3 = []
# risk4 = []
# risk5 = []
# risk6 = []
# # 年份跟公司也append，方便確認每家公司的風險分數沒有對應錯誤
# year_from_news_score = []
# company_from_news_score = []

# for i in range(len(df)):
#     s = df['年月'].iloc[i]
#     match = re.search('\d+/\d+/\d+', s)
#     date = datetime.datetime.strptime(match.group(0), '%Y/%m/%d').date()
#     year = date.year
#     company = df['公司'].iloc[i]
#     isInsert = False
#     for j in range(len(news_score)):
#         if (year == news_score['Year'].iloc[j]) and (company == news_score['company'].iloc[j]):
#             risk1.append(np.float64(news_score['0'].iloc[j]))
#             risk2.append(np.float64(news_score['1'].iloc[j]))
#             risk3.append(np.float64(news_score['2'].iloc[j]))
#             risk4.append(np.float64(news_score['3'].iloc[j]))
#             risk5.append(np.float64(news_score['4'].iloc[j]))
#             risk6.append(np.float64(news_score['5'].iloc[j]))
#             year_from_news_score.append(news_score['Year'].iloc[j])
#             company_from_news_score.append(news_score['company'].iloc[j])
#             isInsert = True
#     if not isInsert:
#         risk1.append(0.0)
#         risk2.append(0.0)
#         risk3.append(0.0)
#         risk4.append(0.0)
#         risk5.append(0.0)
#         risk6.append(0.0)
#         year_from_news_score.append(0)
#         company_from_news_score.append('NO')

In [71]:
# # M106陳柏霖寫法

# df['財務風險'] = risk1
# df['管理風險'] = risk2
# df['法律風險'] = risk3
# df['供應鏈風險'] = risk4
# df['誠信風險'] = risk5
# df['勞動風險'] = risk6
# df['year'] = year_from_news_score
# df['company'] = company_from_news_score
# df.tail()

In [72]:
df["c_0"] = 0.0
df["c_1"] = 0.0
df["c_2"] = 0.0
df["c_3"] = 0.0
df["c_4"] = 0.0
df.tail(2)

,公司,年月,TCRI,總分,CFO,CFO不一致,利息支出率,資產報酬%,營業利益%,借款息%,平均收款天增減,平均售貨天增減,借款依存度%,營收,TSE產業別,c_0,c_1,c_2,c_3,c_4
2286,9943 好樂迪,2017/12/29,5.0,711.0,5.20,0,0.00,15.93,17.41,6.53,1.0,0.0,0.0,29.0,16,0.0,0.0,0.0,0.0,0.0
2287,9946 三發地產,2017/12/29,7.0,243.0,-8.71,1,7.49,3.83,20.42,2.51,9.0,1437.0,102.0,17.0,14,0.0,0.0,0.0,0.0,0.0


In [73]:
company_list = news_score["company"].astype(str).unique()  # 把欄位unique的值都挑出來變成numpy
print(len(company_list))
company_list[:10]

240


array(['1101 台泥', '1201 味全', '1210 大成', '1215 卜蜂', '1216 統一', '1218 泰山',
       '1220 台榮', '1227 佳格', '1232 大統益', '1233 天仁'], dtype=object)

In [74]:
# news_score表格 取出"company" contains"三發地產" 並且 "date" contains 2017的表格
df_temp = news_score.loc[(news_score["company"].str.contains("三發地產"))
                          & (news_score["date"].astype(str).str.contains( str(2017) ))]
df_temp

,company,date,c_0,c_1,c_2,c_3,c_4
427,9946 三發地產,2017,1.0,1.433146e-20,1.467837e-13,2.940595e-25,2.578725e-17


In [75]:
# df_temp c_0欄位取值 用sum是因為有時候會遇到df_temp表格是空的
df_temp["c_0"].sum()  

1.0

In [76]:
# df表格 取出"company" contains"三發地產" 並且 "date" contains 2017的表格
df.loc[(df["公司"].astype(str).str.contains("三發地產") )
                      & (df["年月"].astype(str).str.contains( str(2017) ))]

,公司,年月,TCRI,總分,CFO,CFO不一致,利息支出率,資產報酬%,營業利益%,借款息%,平均收款天增減,平均售貨天增減,借款依存度%,營收,TSE產業別,c_0,c_1,c_2,c_3,c_4
2287,9946 三發地產,2017/12/29,7.0,243.0,-8.71,1,7.49,3.83,20.42,2.51,9.0,1437.0,102.0,17.0,14,0.0,0.0,0.0,0.0,0.0


In [77]:
# df表格 取出"company" contains"三發地產" 並且 "date" contains 2017的表格 的 c_0欄位 = df_temp["c_0"].sum()
df.loc[(df["公司"].astype(str).str.contains("三發地產") )
                      & (df["年月"].astype(str).str.contains( str(2017) )), "c_0"] = df_temp["c_0"].sum()
df.tail(2)

,公司,年月,TCRI,總分,CFO,CFO不一致,利息支出率,資產報酬%,營業利益%,借款息%,平均收款天增減,平均售貨天增減,借款依存度%,營收,TSE產業別,c_0,c_1,c_2,c_3,c_4
2286,9943 好樂迪,2017/12/29,5.0,711.0,5.20,0,0.00,15.93,17.41,6.53,1.0,0.0,0.0,29.0,16,0.0,0.0,0.0,0.0,0.0
2287,9946 三發地產,2017/12/29,7.0,243.0,-8.71,1,7.49,3.83,20.42,2.51,9.0,1437.0,102.0,17.0,14,1.0,0.0,0.0,0.0,0.0


In [78]:
# # 用pandas兩個欄位的for loop應該比較快 但我懶惰重新寫
# for company_name, year in tqdm(news_score[["company", "date"]].to_numpy(), total=428):
#     print(company_name, year)

In [79]:
# 把上面寫成for loop
# for 240公司名list:
#     for 2008-2017年:

for company_name in tqdm(company_list):
    for year in range(2008,2018):
        company_name = str(company_name)
        
        # news_score 取出 company欄位 contains 公司名 & date欄位 contains 年分 
        df_temp = news_score.loc[(news_score["company"].str.contains(company_name))
                       & (news_score["date"].astype(str).str.contains( str(year) ))]


        # 將 c_0 ~ c_4 欄位 
        # 按照 公司名 年分 填入 df 的 c_0 ~ c_4 欄位表格
        df.loc[(df["公司"].astype(str).str.contains(company_name) )
                    & (df["年月"].astype(str).str.contains( str(year) )), "c_0"] = df_temp["c_0"].sum()
        df.loc[(df["公司"].astype(str).str.contains(company_name) )
                    & (df["年月"].astype(str).str.contains( str(year) )), "c_1"] = df_temp["c_1"].sum()
        df.loc[(df["公司"].astype(str).str.contains(company_name) )
                    & (df["年月"].astype(str).str.contains( str(year) )), "c_2"] = df_temp["c_2"].sum()
        df.loc[(df["公司"].astype(str).str.contains(company_name) )
                    & (df["年月"].astype(str).str.contains( str(year) )), "c_3"] = df_temp["c_3"].sum()
        df.loc[(df["公司"].astype(str).str.contains(company_name) )
                    & (df["年月"].astype(str).str.contains( str(year) )), "c_4"] = df_temp["c_4"].sum()

df.tail(2)

100%|██████████| 240/240 [00:37<00:00,  6.48it/s]


,公司,年月,TCRI,總分,CFO,CFO不一致,利息支出率,資產報酬%,營業利益%,借款息%,平均收款天增減,平均售貨天增減,借款依存度%,營收,TSE產業別,c_0,c_1,c_2,c_3,c_4
2286,9943 好樂迪,2017/12/29,5.0,711.0,5.20,0,0.00,15.93,17.41,6.53,1.0,0.0,0.0,29.0,16,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
2287,9946 三發地產,2017/12/29,7.0,243.0,-8.71,1,7.49,3.83,20.42,2.51,9.0,1437.0,102.0,17.0,14,1.0,1.433146e-20,1.467837e-13,2.940595e-25,2.578725e-17


### 因為有些年有些公司沒有文章 看機率全為0.0的有幾筆

In [80]:
# 把 c_0~c_4都等於0 的index 挑出來
index_to_drop = df[(df['c_0'] == 0.0) &
                   (df['c_1'] == 0.0) &
                   (df['c_2'] == 0.0) &
                   (df['c_3'] == 0.0) &
                   (df['c_4'] == 0.0)].index     # 把 c_0~c_4都等於0 的 row的 index 挑出來
print(len(index_to_drop))
index_to_drop

1872


Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            2275, 2276, 2277, 2278, 2279, 2280, 2283, 2284, 2285, 2286],
           dtype='int64', length=1872)

In [81]:
df2 = df.drop(index=index_to_drop, axis=1)       # 刪除指定index 整個row
df2 = df2.reset_index(drop=True)           # index重新排序
print(df2.shape)
del df2

(416, 20)


In [82]:
df.to_csv('data/TCRI_2008_2017_prob.csv', index=False)

# 2 Feature Engineering

## 2-1 根據基本總分產生的初步TCRI等級

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas()

/home/barry/miniconda3/envs/bertenv/lib/python3.7/site-packages/tqdm/std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
df = pd.read_csv("data/TCRI_2008_2017_prob.csv", encoding="utf-8")
df["c_0"] = df["c_0"].astype(float)  # 讀取後將欄位弄成float
df["c_1"] = df["c_1"].astype(float)
df["c_2"] = df["c_2"].astype(float)
df["c_3"] = df["c_3"].astype(float)
df["c_4"] = df["c_4"].astype(float)
df["TCRI"] = df["TCRI"].astype(float)
print(df.shape)
df.tail(2)

(2288, 20)


,公司,年月,TCRI,總分,CFO,CFO不一致,利息支出率,資產報酬%,營業利益%,借款息%,平均收款天增減,平均售貨天增減,借款依存度%,營收,TSE產業別,c_0,c_1,c_2,c_3,c_4
2286,9943 好樂迪,2017/12/29,5.0,711.0,5.20,0,0.00,15.93,17.41,6.53,1.0,0.0,0.0,29.0,16,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
2287,9946 三發地產,2017/12/29,7.0,243.0,-8.71,1,7.49,3.83,20.42,2.51,9.0,1437.0,102.0,17.0,14,1.0,1.433146e-20,1.467837e-13,2.940595e-25,2.578725e-17


In [3]:
def transformBasicTCRILevel(value):
    level = 0
    if value < 165.0:
        level = 9
    elif value < 250.0:
        level = 8
    elif value < 335.0:
        level = 7
    elif value < 420.0:
        level = 6
    elif value < 505.0:
        level = 5
    elif value < 590.0:
        level = 4
    elif value < 675.0:
        level = 3
    elif value < 760.0:
        level = 2
    else:
        level = 1
    return level

In [4]:
basicTCRI = []
for score in df['總分']:
    basicTCRI.append(transformBasicTCRILevel(score))

In [5]:
from sklearn.metrics import mean_squared_error
from math import sqrt
rmse = sqrt(mean_squared_error(basicTCRI, df['TCRI']))
print(rmse)

2.3138757483972596


In [6]:
# 用初步等級將總分替換
df.drop(['總分'], axis=1, inplace=True)
df['初步等級'] = basicTCRI
df.head()

,公司,年月,TCRI,CFO,CFO不一致,利息支出率,資產報酬%,營業利益%,借款息%,平均收款天增減,平均售貨天增減,借款依存度%,營收,TSE產業別,c_0,c_1,c_2,c_3,c_4,初步等級
0,1101 台泥,2008/12/31,3.0,80.43,0,3.51,2.93,6.64,3.72,-2.0,-2.0,88.0,785.0,1,0.0,0.0,0.0,0.0,0.0,3
1,1201 味全,2008/12/31,5.0,10.75,0,0.54,4.62,4.04,3.27,-2.0,1.0,63.0,202.0,2,0.0,0.0,0.0,0.0,0.0,3
2,1210 大成,2008/12/31,4.0,23.25,0,0.42,5.93,1.96,4.42,-5.0,-9.0,43.0,712.0,2,0.0,0.0,0.0,0.0,0.0,2
3,1215 卜蜂,2008/12/31,4.0,1.77,0,0.27,5.49,2.37,3.73,0.0,-1.0,38.0,154.0,2,0.0,0.0,0.0,0.0,0.0,4
4,1216 統一,2008/12/31,4.0,178.78,0,1.38,4.02,4.95,3.56,-1.0,1.0,125.0,2997.0,2,0.0,0.0,0.0,0.0,0.0,3


## 2-2 門檻等級：規模

In [7]:
thresholdLevel = []
for i in range(len(df)):
    rev = df['營收'].iloc[i]
    level = df['初步等級'].iloc[i]
    threshold = 0
    
    # 營收不到5億，則不列入前5等；電子業門檻較高，門檻為10億
    if df['TSE產業別'].iloc[i] == 28: # 電子零組件業產業別代碼為28
        threshold = 10
    else:
        threshold = 5
    if rev < threshold and level <= 5:
        level = 6
        
    # 營收不到10億，則不列入前4等；電子業門檻較高，門檻為20億
    if df['TSE產業別'].iloc[i] == 28: # 電子零組件業產業別代碼為28
        threshold = 20
    else:
        threshold = 10
    if rev < threshold and level <= 4:
        level = 5
    
    # 營收不到30億，則不列入前3等；電子業門檻較高，門檻為40億
    if df['TSE產業別'].iloc[i] == 28: # 電子零組件業產業別代碼為28
        threshold = 40
    else:
        threshold = 30
    if rev < threshold and level <= 3:
        level = 4
        
    # 營收不到100億，則不列入前2等
    threshold = 100
    if rev < threshold and level <= 2:
        level = 3
        
    # 營收不到200億，則不列入前1等
    threshold = 200
    if rev < threshold and level <= 1:
        level = 2
    
    # 補充電子業門檻較高原因：受景氣影響較深，除非擁有關鍵技術或特殊的利基市場、獨門生意，否則一律適用較嚴格的規模門檻。
    
    thresholdLevel.append(level)

df['門檻等級_規模'] = thresholdLevel
df.head()

,公司,年月,TCRI,CFO,CFO不一致,利息支出率,資產報酬%,營業利益%,借款息%,平均收款天增減,...,借款依存度%,營收,TSE產業別,c_0,c_1,c_2,c_3,c_4,初步等級,門檻等級_規模
0,1101 台泥,2008/12/31,3.0,80.43,0,3.51,2.93,6.64,3.72,-2.0,...,88.0,785.0,1,0.0,0.0,0.0,0.0,0.0,3,3
1,1201 味全,2008/12/31,5.0,10.75,0,0.54,4.62,4.04,3.27,-2.0,...,63.0,202.0,2,0.0,0.0,0.0,0.0,0.0,3,3
2,1210 大成,2008/12/31,4.0,23.25,0,0.42,5.93,1.96,4.42,-5.0,...,43.0,712.0,2,0.0,0.0,0.0,0.0,0.0,2,2
3,1215 卜蜂,2008/12/31,4.0,1.77,0,0.27,5.49,2.37,3.73,0.0,...,38.0,154.0,2,0.0,0.0,0.0,0.0,0.0,4,4
4,1216 統一,2008/12/31,4.0,178.78,0,1.38,4.02,4.95,3.56,-1.0,...,125.0,2997.0,2,0.0,0.0,0.0,0.0,0.0,3,3


## 2-2 APL 財務壓力計

### 2-2-1 Assets Management

In [8]:
# # 確認有無離群值
# plt.subplot(1, 2, 1)
# plt.boxplot(df['平均收款天增減'])
# plt.subplot(1, 2, 2)
# plt.boxplot(df['平均售貨天增減'])
# plt.tight_layout()

In [9]:
from scipy import stats
index_temp_1 = df[(np.abs(stats.zscore(df['平均收款天增減'])) > 3)].index
index_temp_2 = df[(np.abs(stats.zscore(df['平均售貨天增減'])) > 3)].index
# display(index_temp_1)
# display(index_temp_2)

# 聯集：讓index不重複
def find_union(lst1, lst2):
    final_list = list(set(lst1) | set(lst2)) 
    return final_list

index_to_drop = find_union(index_temp_1, index_temp_2)

df.drop(index=index_to_drop, axis=1, inplace=True)
df.head(2)

,公司,年月,TCRI,CFO,CFO不一致,利息支出率,資產報酬%,營業利益%,借款息%,平均收款天增減,...,借款依存度%,營收,TSE產業別,c_0,c_1,c_2,c_3,c_4,初步等級,門檻等級_規模
0,1101 台泥,2008/12/31,3.0,80.43,0,3.51,2.93,6.64,3.72,-2.0,...,88.0,785.0,1,0.0,0.0,0.0,0.0,0.0,3,3
1,1201 味全,2008/12/31,5.0,10.75,0,0.54,4.62,4.04,3.27,-2.0,...,63.0,202.0,2,0.0,0.0,0.0,0.0,0.0,3,3


In [10]:
# plt.subplot(1, 2, 1)
# plt.boxplot(df['平均收款天增減'])
# plt.subplot(1, 2, 2)
# plt.boxplot(df['平均售貨天增減'])
# plt.tight_layout()

In [11]:
# A1. 平均收帳天數比前一期慢 >= 20 天
# df[df['平均收款天增減'] >= 20]
A1 = []
for d in df['平均收款天增減']:
    if d >= 20:
        A1.append(int(d / 20))
    else:
        A1.append(0)
df['A1'] = A1

In [12]:
# A3. 平均售貨天數比前一期慢 >= 20 天
A3 = []
for d in df['平均售貨天增減']:
    if d >= 20:
        A3.append(int(d / 20))
    else:
        A3.append(0)
df['A3'] = A3

### 2-2-2 Profitability

In [13]:
# P1. 當期的總資產報酬率(ROA)不敷借款息
# df[df['資產報酬%'] - df['借款息%'] <= 0]
P1 = []
for i in range(len(df)):
    if df['資產報酬%'].iloc[i] - df['借款息%'].iloc[i] <= 0:
        P1.append(True)
    else:
        P1.append(False)

In [14]:
# P2. 當期的營業利益率(OP%)不足支應利息支出率(INT%)
# df[df['營業利益%'] - df['利息支出率'] <= 0]
P2 = []
for i in range(len(df)):
    if df['營業利益%'].iloc[i] - df['利息支出率'].iloc[i] <= 0:
        P2.append(True)
    else:
        P2.append(False)

In [15]:
'''
P發生的case
(1) P1且P2
(2) 連續3期P1（先略過）
'''
P_case1 = []
for i in range(len(df)):
    if P1[i] and P2[i]:
        P_case1.append(1)
    else:
        P_case1.append(0)
df['P'] = P_case1
df.head()

,公司,年月,TCRI,CFO,CFO不一致,利息支出率,資產報酬%,營業利益%,借款息%,平均收款天增減,...,c_0,c_1,c_2,c_3,c_4,初步等級,門檻等級_規模,A1,A3,P
0,1101 台泥,2008/12/31,3.0,80.43,0,3.51,2.93,6.64,3.72,-2.0,...,0.0,0.0,0.0,0.0,0.0,3,3,0,0,0
1,1201 味全,2008/12/31,5.0,10.75,0,0.54,4.62,4.04,3.27,-2.0,...,0.0,0.0,0.0,0.0,0.0,3,3,0,0,0
2,1210 大成,2008/12/31,4.0,23.25,0,0.42,5.93,1.96,4.42,-5.0,...,0.0,0.0,0.0,0.0,0.0,2,2,0,0,0
3,1215 卜蜂,2008/12/31,4.0,1.77,0,0.27,5.49,2.37,3.73,0.0,...,0.0,0.0,0.0,0.0,0.0,4,4,0,0,0
4,1216 統一,2008/12/31,4.0,178.78,0,1.38,4.02,4.95,3.56,-1.0,...,0.0,0.0,0.0,0.0,0.0,3,3,0,0,0


### 2-2-3 Liquidity

In [16]:
# L1. 借款依存度(DE%)超過100%
# df[df['借款依存度%'] > 100]
L1 = []
for l in df['借款依存度%']:
    if l > 100:
        L1.append(1)
    else:
        L1.append(0)
df['L1'] = L1

,公司,年月,TCRI,CFO,CFO不一致,利息支出率,資產報酬%,營業利益%,借款息%,平均收款天增減,...,c_1,c_2,c_3,c_4,初步等級,門檻等級_規模,A1,A3,P,L1
0,1101 台泥,2008/12/31,3.0,80.43,0,3.51,2.93,6.64,3.72,-2.0,...,0.0,0.0,0.0,0.0,3,3,0,0,0,0
1,1201 味全,2008/12/31,5.0,10.75,0,0.54,4.62,4.04,3.27,-2.0,...,0.0,0.0,0.0,0.0,3,3,0,0,0,0
2,1210 大成,2008/12/31,4.0,23.25,0,0.42,5.93,1.96,4.42,-5.0,...,0.0,0.0,0.0,0.0,2,2,0,0,0,0
3,1215 卜蜂,2008/12/31,4.0,1.77,0,0.27,5.49,2.37,3.73,0.0,...,0.0,0.0,0.0,0.0,4,4,0,0,0,0
4,1216 統一,2008/12/31,4.0,178.78,0,1.38,4.02,4.95,3.56,-1.0,...,0.0,0.0,0.0,0.0,3,3,0,0,0,1


In [17]:
# L3. CFO為負，不足資金 < -0.25億
# df[df['CFO'] < -0.25]
L3 = []
for l in df['CFO']:
    if l < 0.25:
        L3.append(1)
    else:
        L3.append(0)
df['L3'] = L3

In [18]:
df.head()

,公司,年月,TCRI,CFO,CFO不一致,利息支出率,資產報酬%,營業利益%,借款息%,平均收款天增減,...,c_2,c_3,c_4,初步等級,門檻等級_規模,A1,A3,P,L1,L3
0,1101 台泥,2008/12/31,3.0,80.43,0,3.51,2.93,6.64,3.72,-2.0,...,0.0,0.0,0.0,3,3,0,0,0,0,0
1,1201 味全,2008/12/31,5.0,10.75,0,0.54,4.62,4.04,3.27,-2.0,...,0.0,0.0,0.0,3,3,0,0,0,0,0
2,1210 大成,2008/12/31,4.0,23.25,0,0.42,5.93,1.96,4.42,-5.0,...,0.0,0.0,0.0,2,2,0,0,0,0,0
3,1215 卜蜂,2008/12/31,4.0,1.77,0,0.27,5.49,2.37,3.73,0.0,...,0.0,0.0,0.0,4,4,0,0,0,0,0
4,1216 統一,2008/12/31,4.0,178.78,0,1.38,4.02,4.95,3.56,-1.0,...,0.0,0.0,0.0,3,3,0,0,0,1,0


# 3 訓練前的準備

## 3-1 Resample imbalanced labels
https://www.kaggle.com/rafjaa/resampling-strategies-for-imbalanced-datasets

In [19]:
df['TCRI'].value_counts()

4.0    509
5.0    440
6.0    398
7.0    270
3.0    206
8.0    168
9.0    102
0.0     80
2.0     45
1.0     40
Name: TCRI, dtype: int64

In [20]:
# df['TCRI'].value_counts().plot(kind='bar')

In [21]:
# Class count
count_TCRI_4 = 2*len(df[df['TCRI'] == 4])


# Divide by class
df_TCRI_0 = df[df['TCRI'] == 0.0]
df_TCRI_1 = df[df['TCRI'] == 1]
df_TCRI_2 = df[df['TCRI'] == 2]
df_TCRI_3 = df[df['TCRI'] == 3]
df_TCRI_4 = df[df['TCRI'] == 4]
df_TCRI_5 = df[df['TCRI'] == 5]
df_TCRI_6 = df[df['TCRI'] == 6]
df_TCRI_7 = df[df['TCRI'] == 7]
df_TCRI_8 = df[df['TCRI'] == 8]
df_TCRI_9 = df[df['TCRI'] == 9]

In [22]:
# 把每個類別都複製到跟4類別一樣多

df_TCRI_0_over = df_TCRI_0.sample(n=count_TCRI_4, replace=True, random_state=2018, )
df_TCRI_1_over = df_TCRI_1.sample(n=count_TCRI_4, replace=True, random_state=2018, )
df_TCRI_2_over = df_TCRI_2.sample(n=count_TCRI_4, replace=True, random_state=2018, )
df_TCRI_3_over = df_TCRI_3.sample(n=count_TCRI_4, replace=True, random_state=2018, )
df_TCRI_5_over = df_TCRI_5.sample(n=count_TCRI_4, replace=True, random_state=2018, )
df_TCRI_6_over = df_TCRI_6.sample(n=count_TCRI_4, replace=True, random_state=2018, )
df_TCRI_7_over = df_TCRI_7.sample(n=count_TCRI_4, replace=True, random_state=2018, )
df_TCRI_8_over = df_TCRI_8.sample(n=count_TCRI_4, replace=True, random_state=2018, )
df_TCRI_9_over = df_TCRI_9.sample(n=count_TCRI_4, replace=True, random_state=2018, )

In [23]:
# 新的複製的data 合併
df_resample = pd.concat([df_TCRI_0_over, df_TCRI_1_over, df_TCRI_2_over, df_TCRI_3_over, df_TCRI_4, df_TCRI_5_over, df_TCRI_6_over, df_TCRI_7_over, df_TCRI_8_over, df_TCRI_9_over])
df_resample['TCRI'].value_counts()

9.0    1018
8.0    1018
7.0    1018
6.0    1018
5.0    1018
3.0    1018
2.0    1018
1.0    1018
0.0    1018
4.0     509
Name: TCRI, dtype: int64

In [24]:
# df_resample['TCRI'].value_counts().plot(kind='bar')

## 3-2 資料前處理 function

In [25]:
def dataPreprocessing(df, IsSelected, HasCrisis, GenMoreData):
    # 3-1 區分Feature (X) 跟Label (Y)
    Y = df['TCRI'].values
    
    if HasCrisis:
        X = df.drop(['TCRI', '公司', '年月'], axis = 1).values
    else:
        X = df.drop(['TCRI', '公司', '年月', 'c_0', 'c_1', 'c_2', 'c_3', 'c_4' ], axis = 1).values
    
    if IsSelected: # Feature Selection with Random Forest
        clf = ExtraTreesClassifier(n_estimators=50)
        clf = clf.fit(X, Y)
        model = SelectFromModel(clf, prefit=True)
        X = model.transform(X)
        X.shape
    
    # 3-2 標準化
    minmax_scale = preprocessing.MinMaxScaler(feature_range=(0, 1))
    X_minmax = minmax_scale.fit_transform(X)
    X_minmax.shape
    
    # 3-3 Label要轉成二進位的matrix，才能使用categorical_crossentropy
    from tensorflow.keras.utils import to_categorical
    y = to_categorical(Y)
    
    # 3-4 Train Test Split
    X_train, X_test, y_train, y_test = train_test_split(X_minmax, y, test_size=0.2, random_state=42)
    
    # 3-5 拿training data的10%作為Validation Split
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)
    
    # 3-5 產生更多data
    for i in range(GenMoreData):
        X_train = np.append(X_train, X_train, axis=0)
        X_test = np.append(X_test, X_test, axis=0)
        X_val = np.append(X_val, X_val, axis=0)
        y_train = np.append(y_train, y_train, axis=0)
        y_test = np.append(y_test, y_test, axis=0)
        y_val = np.append(y_val, y_val, axis=0)
    
    return X_train, X_test, y_train, y_test, X_val, y_val

# 4 訓練MLP模型

In [26]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel, TFBertForSequenceClassification, AdamW

from tensorflow import keras
from tensorflow.keras import Sequential, Model, Input
from tensorflow.keras.layers import Dense, Embedding, SpatialDropout1D, Dropout, Activation, Flatten, InputLayer
from sklearn import preprocessing
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, confusion_matrix
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [27]:
# # 如果GPU被占用
# config = tf.ConfigProto(
#         device_count = {'GPU': 0}
#     )
# session = tf.Session(config=config)

In [28]:
def MLP_model(INPUT_DIM, OUTPUT_DIM):
    model = Sequential()
    model.add(Dense(units=64, input_dim=INPUT_DIM, kernel_initializer='uniform', activation='relu'))
    model.add(Dense(units=32, activation='relu'))
    model.add(Dropout(0.2))
#     model.add(Dense(16, activation='relu'))
#     model.add(Dropout(0.2))
    model.add(Dense(units=OUTPUT_DIM, activation='softmax'))
    model.summary()
    
    return model

In [29]:
def show_train_history(train_history, train_key, val_key):
    plt.plot(train_history.history[train_key])
    plt.plot(train_history.history[val_key])
    plt.title('Model accuracy' if 'acc' in train_key else 'Model loss')
    plt.ylabel('accuracy' if 'acc' in train_key else 'loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.plot()

In [30]:
X_train, X_test, y_train, y_test, X_val, y_val = dataPreprocessing(df_resample, IsSelected=False, HasCrisis=True, GenMoreData=6)

In [31]:
display(X_train.shape[1])

23

In [32]:
%%time
model = MLP_model(X_train.shape[1], y_train.shape[1])
# EPOCHS = 100
# LR = 0.1
# DECAY = LR / EPOCHS
# MOMENTUM = 0.8
# SGD = SGD(lr=LR, momentum=MOMENTUM, decay=DECAY, nesterov=False)
# ADAM = Adam(lr=LR, decay=DECAY)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])
history = model.fit(x=X_train, y=y_train, validation_data=(X_val, y_val), epochs=25, batch_size=16, verbose=1)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                1536      
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                330       
Total params: 3,946
Trainable params: 3,946
Non-trainable params: 0
_________________________________________________________________
Train on 445568 samples, validate on 49536 samples
Epoch 1/25
445568/445568 [==============================] - 33s 74us/sample - loss: 0.9946 - categorical_accuracy: 0.5905 - val_loss: 0.7811 - val_categorical_accuracy: 0.6835
Epoch 2/25
445568/445568 

In [42]:
# model.save('model/model_TCRI.h5')

In [ ]:
# tf.keras.models.save_model(
#     model,
#     "model/model_TCRI",
#     overwrite=True,
#     include_optimizer=True,
# ) 

In [ ]:
# load_model = tf.keras.models.load_model('model/model_TCRI')
# load_model.summary()

In [37]:
X_train_RF, X_test_RF, y_train_RF, y_test_RF, X_val_RF, y_val_RF = dataPreprocessing(df_resample, IsSelected=False, HasCrisis=False, GenMoreData=6)

In [38]:
display(X_train_RF.shape[1])

18

In [41]:
%%time
model_RF = MLP_model(X_train_RF.shape[1], y_train_RF.shape[1])
# OPTIMIZER = Adam(lr=0.001)
model_RF.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])
history_RF = model_RF.fit(x=X_train_RF, y=y_train_RF, validation_data=(X_val_RF, y_val_RF), epochs=25, batch_size=16, verbose=1)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 64)                1216      
_________________________________________________________________
dense_7 (Dense)              (None, 32)                2080      
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 10)                330       
Total params: 3,626
Trainable params: 3,626
Non-trainable params: 0
_________________________________________________________________
Train on 445568 samples, validate on 49536 samples
Epoch 1/25
445568/445568 [==============================] - 32s 72us/sample - loss: 1.0647 - categorical_accuracy: 0.5577 - val_loss: 0.9000 - val_categorical_accuracy: 0.6202
Epoch 2/25
445568/44556

In [51]:
# model_RF.save('Model/TCRI/MLP_64units_32units_0513_withoutsent.h5')

# 5 評估模型與預測

## 5-1 載入模型

In [52]:
# # 載入先前已經訓練好的模型
# # model = load_model('Model/TCRI/MLP_64units_32units_0513_sentiment.h5')
# best_model = load_model('Model/TCRI/MLP_64units_32units_0513_sentiment.h5')

## 5-2 評估模型

### 5-2-1 有輿情

In [ ]:
# # show_train_history(history, 'categorical_accuracy', 'val_categorical_accuracy')
# scores = model.evaluate(X_test, y_test)
# print(scores)

In [33]:
y_pred = model.predict_classes(X_test)
y_pred

array([5, 0, 1, ..., 2, 2, 1])

In [34]:
y_test_classes = np.argmax(y_test, axis=1)
y_test_classes

array([5, 0, 1, ..., 2, 2, 1])

In [35]:
# np.argmax(y_test, axis=1)

In [36]:
print(accuracy_score(y_test_classes, y_pred))
print(confusion_matrix(y_test_classes, y_pred))
print(classification_report(y_test_classes, y_pred))

0.8713178294573644
[[12992     0     0     0     0     0     0     0     0     0]
 [    0 13376     0     0     0     0     0     0     0     0]
 [    0     0 13248     0     0     0     0     0     0     0]
 [    0     0   256 12352   128    64     0     0     0     0]
 [    0     0    64  1664  2944  1856   384   320     0     0]
 [    0     0     0   384  1152  9408  1792   192     0     0]
 [    0     0     0    64    64  1600  9344   960   320     0]
 [    0     0     0     0     0   768  1152  9920  1344    64]
 [    0     0     0     0     0   128   640   192 12544   128]
 [  256     0     0     0     0     0     0     0     0 11776]]
              precision    recall  f1-score   support

           0       0.98      1.00      0.99     12992
           1       1.00      1.00      1.00     13376
           2       0.98      1.00      0.99     13248
           3       0.85      0.96      0.91     12800
           4       0.69      0.41      0.51      7232
           5       0.68  

In [ ]:
# sns.heatmap(confusion_matrix(np.argmax(y_test, axis=1), y_pred), cmap='YlGnBu')

In [64]:
# pd.DataFrame(confusion_matrix(np.argmax(y_test, axis=1), y_pred)).to_csv('TCRI/ConfusionMatrix_Sentiment.csv')

### 5-2-2 沒有輿情

In [65]:
# 載入先前已經訓練好的模型
# model_RF = load_model('Model/TCRI/MLP_64units_32units_0513_withoutsent.h5')

In [ ]:
# # show_train_history(history_RF, 'categorical_accuracy', 'val_categorical_accuracy')
# scores_RF = model_RF.evaluate(X_test_RF, y_test_RF)
# print(scores_RF)

In [46]:
y_pred_RF = model_RF.predict_classes(X_test_RF)
y_pred_RF

array([3, 0, 1, ..., 2, 2, 1])

In [47]:
y_test_RF_classes = np.argmax(y_test_RF, axis=1)
y_test_RF_classes

array([5, 0, 1, ..., 2, 2, 1])

In [48]:
print(accuracy_score(y_test_RF_classes, y_pred_RF))
print(confusion_matrix(y_test_RF_classes, y_pred_RF))
print(classification_report(y_test_RF_classes, y_pred_RF))

0.8101291989664083
[[12992     0     0     0     0     0     0     0     0     0]
 [    0 13376     0     0     0     0     0     0     0     0]
 [    0     0 13248     0     0     0     0     0     0     0]
 [    0     0   128 12288   256   128     0     0     0     0]
 [    0     0   128  1728  2432  2304   576    64     0     0]
 [    0     0     0   640   256  9344  2304   384     0     0]
 [  128     0     0   128   256  2304  8128  1024   256   128]
 [   64     0     0     0     0  1088  1088  9152  1600   256]
 [   64     0     0     0     0     0   320    64 13184     0]
 [  704     0     0     0     0     0     0     0   192 11136]]
              precision    recall  f1-score   support

           0       0.93      1.00      0.96     12992
           1       1.00      1.00      1.00     13376
           2       0.98      1.00      0.99     13248
           3       0.83      0.96      0.89     12800
           4       0.76      0.34      0.47      7232
           5       0.62  

In [ ]:
# sns.heatmap(confusion_matrix(np.argmax(y_test_RF, axis=1), y_pred_RF), cmap='YlGnBu')

In [75]:
# pd.DataFrame(confusion_matrix(np.argmax(y_test_RF, axis=1), y_pred_RF)).to_csv('TCRI/ConfusionMatrix_WithoutSent.csv')